In [2]:
import pandas as pd
import numpy as np
"""import plotly.offline as py
from plotly.subplots import make_subplots
import plotly.graph_objs as go"""
import plotly.express as px

df = pd.read_csv("2020_opendatascience_poll_data.csv")

In [2]:
df.head()

,Timestamp,What is your residence country (where are you from?),Your timezone (we need that to schedule post timing better),Education,Work status,Data Science expertise level,Age,Are you satisfied with channel update frequency?,Are you satisfied with channel's material complexity?,What field are you interested in (multiple choices are possible),How did you find out about the channel?,How likely are you going to recommend a channel to your friend or colleague?,What’s the main reason for your score? *,"If you want to reach for the editors and to write something, please use the field below:"
0,2020/01/29 1:30:57 PM GMT+3,Ukraine,GMT+3,Undergrad,Student + part time remote job,Middle,18-24,"Yes, it's about perfect",It's all ok,#CV #DL #imageprocessing #videolearning;#RL #D...,Forward from a friend,5,All stuff is absolutely brilliant! Thank you f...,NaN
1,2020/01/29 1:31:19 PM GMT+3,Russia,GMT+3,Graduate,Employed,Middle,31-42,"Nope, less frequent posting will be all right ...",It's all ok,#RL #DL;#NLP #NLU #conversational #dialoguesys...,"It's been so long time ago, I can't remember (...",4,it's ok,"post some jobs with salary ranges, especially ..."
2,2020/01/29 1:32:48 PM GMT+3,Ukraine,GMT+2,PhD,Unemployed,"Novice (Studying courses, active learning)",25-30,"Yes, it's about perfect",Need more specific and complicated materials,#WhereToStart #EntryLevel #Novice #MOOC #Learn...,Telegram channel search,3,NaN,NaN
3,2020/01/29 1:33:27 PM GMT+3,Italy,GMT+1,"No degree at all, still learning / self-taught",Student,"Novice (Studying courses, active learning)",18-24,"Yes, it's about perfect",It's all ok,#CV #DL #imageprocessing #videolearning;#RL #D...,Forward from a friend,5,Mainly due to material shared,NaN
4,2020/01/29 1:33:49 PM GMT+3,Ukraine,GMT+2,Graduate,Employed,Middle,18-24,"Yes, it's about perfect",It's all ok,#RL #DL;#NLP #NLU #conversational #dialoguesys...,"It's been so long time ago, I can't remember (...",2,It's not super useful actually. Good enough to...,NaN


In [15]:
df.columns

Index(['Timestamp', 'What is your residence country (where are you from?)',
       'Your timezone (we need that to schedule post timing better)',
       'Education', 'Work status', 'Data Science expertise  level', 'Age',
       'Are you satisfied with channel update frequency?',
       'Are you satisfied with channel's material complexity?',
       'What field are you interested in (multiple choices are possible)',
       'How did you find out about the channel?',
       'How likely are you going to recommend a channel to your friend or colleague?',
       'What’s the main reason for your score? *',
       'If you want to reach for the editors and to write something, please use the field below:',
       'Timezone', 'Country', 'Work', 'Experience', 'Sat_update',
       'Sat_material', 'Interests', 'How_found', 'Recommend', 'Why'],
      dtype='object')

In [3]:
df["Country"] = df.iloc[:,1]
df["Timezone"] = df.iloc[:,2]
df["Work"] = df.iloc[:,4]
df["Experience"] = df.iloc[:,5]
df["Sat_update"] = df.iloc[:,7]
df["Sat_material"] = df.iloc[:,8]
df["Interests"] = df.iloc[:,9]
df["How_found"] = df.iloc[:,10]
df["Recommend"] = df.iloc[:,11]
df["Why"] = df.iloc[:,12]
df = df.drop(["Your timezone (we need that to schedule post timing better)",
             "What is your residence country (where are you from?)",
             "Work status",
             "Data Science expertise  level",
             "Are you satisfied with channel update frequency?",
             "Are you satisfied with channel's material complexity?",
             "What field are you interested in (multiple choices are possible)",
             "How did you find out about the channel?",
             "How likely are you going to recommend a channel to your friend or colleague?",
             "What’s the main reason for your score? *"],axis=1, inplace=False)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 14 columns):
 #   Column                                                                                    Non-Null Count  Dtype 
---  ------                                                                                    --------------  ----- 
 0   Timestamp                                                                                 557 non-null    object
 1   Education                                                                                 557 non-null    object
 2   Age                                                                                       557 non-null    object
 3   If you want to reach for the editors and to write something, please use the field below:  96 non-null     object
 4   Country                                                                                   548 non-null    object
 5   Timezone                                                        

In [5]:
df.Education.unique()

array(['Undergrad', 'Graduate', 'PhD',
       'No degree at all, still learning / self-taught',
       "I hold a degree in a field irrelevant to DS but as it comes to DS, I'm all self-taught",
       'Post-Doc'], dtype=object)

Data visualization is a very interesting subject, very creative and subjective. But there are some common principles that drive the work of every DataScientists, and the most important are:
- Clarity
- Interpretability

We will try to present some common techniques to represent data, more over we will try to show how different types of plots or data manipulation can make a plot more interpretable

# 01 Countries

Let's start to see where the audience come from. We start by aggregating rows via the groupby operation provided by pandas.
Once groups are created we call .size() to get how many rows belong to each group. Than we create a DataFrame that will be fed to plotly to create the plot. Plotly works in this way:
- Provide the dataFrame
- Provide columns of the DataFrame that will be plotted

(Actually Plotly can work also with standard arrays, but here we will use always DataFrames)

In [39]:
countries = pd.DataFrame(data= df.groupby("Country").size(), columns=["Count"])
countries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56 entries, Argentina to Yemen
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Count   56 non-null     int64
dtypes: int64(1)
memory usage: 896.0+ bytes


You see that a single column 'Count' was created, while the name of the country represent the index of the DataFrame.
To create a column with names of countries we do the following

In [40]:
countries = countries.reset_index()
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  56 non-null     object
 1   Count    56 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ KB


Now we are ready to use APIs of plotly to create fantastic plots.

Let's start with one of the most common charts, the Pie Chart

In [41]:
fig = px.pie(countries, values='Count', names='Country', title='Countries of the audience')
fig.show()

It contains all information we were looking for, but there is a problem, isn't it?
Due the great fragmentation of countries, It's difficult to see where your country is (in particular if It has few interviewed).
Let's try to fix this using another plot, the Bar Plot

In [73]:
countries.sort_values(by="Country", ascending=True) #sort value by country name
fig = px.bar(countries, y='Count', x='Country', title='Countries of the audience', 
             hover_data=['Count'], color='Count')
fig.update_layout(
    xaxis_tickfont_size=10
)
fig.show()

With this visualization info should be slightly easier to be retrieved. But due to the great difference between Russia and India with the other groups, the plot isn't very beautiful. Let's try to fix this.

Note that if your goal is to show the strong difference between countries, this can be good. But if you only want to present data to an audience with beautiful plots, maybe the following "trick" will be useful.

In [72]:
countries["Count_squared"] = countries["Count"]**(1/3)
fig = px.bar(countries, y='Count_squared', x='Country', title='Countries of the audience', 
             hover_data=['Count'], color='Count')
fig.update_layout(
    xaxis_tickfont_size=10,
    yaxis=dict(
        title='Third root of Count',
        showticklabels=False
    )
)
fig.show()

If we compat data using a squashing function like the square root or the third root (like here) we obtain a slightly more uniform plot. Of course the true numbers are important, in fact thanks to the interactive plot is possible to see the true value for each country (move the mouse over the desired country).
Lables in the y-axis are no more important, because they would represent the third square of the actual values.

If this "trick" is not suitable for your use-case, another solution could be to plot only the K-highest countries.

In [74]:
K = 18
countries_to_plot = countries.sort_values(by="Count", ascending=False)[:K]
fig = px.bar(countries_to_plot, y='Count', x='Country', title='Countries of the audience', 
             hover_data=['Count'], color='Count')
fig.update_layout(
    xaxis_tickfont_size=10
)
fig.show()

# 02 Work status

Then let's analyze the work status of people

In [ ]:
fig = px.pie(df, values='What is your residence country (where are you from?)', names='country', title='Countries of the audience')
fig.show()